# Notebook to check the Quality of data after filtration

**Created by :** Srivalli Kolla

**Created on :** 22 April, 2025

**Modified on :** 24 April, 2025

**University of Würzburg**

Env : scanpy (Python 3.12.2)

# Importing Packages

In [1]:
import bbknn
import anndata
import numpy as np
import pandas as pd
import scanpy as sc
import seaborn as sns
import datetime
import os
from pywaffle import Waffle
import matplotlib.pyplot as plt

In [2]:
sc.settings.verbosity = 3
sc.logging.print_versions()

plt.rcParams['figure.dpi'] = 300  
plt.rcParams['savefig.dpi'] = 300

timestamp = datetime.datetime.now().strftime("%d_%m_%y")

-----
anndata     0.11.3
scanpy      1.10.4
-----
Cython                      3.0.12
PIL                         11.1.0
annoy                       NA
anyio                       NA
arrow                       1.3.0
asttokens                   NA
attr                        25.1.0
attrs                       25.1.0
babel                       2.17.0
bbknn                       1.6.0
certifi                     2025.01.31
cffi                        1.17.1
charset_normalizer          3.4.1
colorama                    0.4.6
comm                        0.2.2
cycler                      0.12.1
cython                      3.0.12
cython_runtime              NA
dateutil                    2.9.0.post0
debugpy                     1.8.12
decorator                   5.2.1
defusedxml                  0.7.1
executing                   2.1.0
fastjsonschema              NA
fqdn                        NA
h5py                        3.13.0
idna                        3.10
igraph                      0.

/home/gruengroup/srivalli/miniconda3/envs/scanpy/lib/python3.13/site-packages/session_info/main.py:213: UserWarning: The '__version__' attribute is deprecated and will be removed in MarkupSafe 3.1. Use feature detection, or `importlib.metadata.version("markupsafe")`, instead.
  mod_version = _find_version(mod.__version__)


# Import Data

In [3]:
adata = sc.read_h5ad('./Github/ACM_sn_2025/data/acm_raw_basic_qc_filtered_22_04_25.h5ad')
adata

AnnData object with n_obs × n_vars = 157012 × 32285
    obs: 'sample', 'Sample_Name', 'Sex', 'Genotype', 'Treatment', 'Condition', 'Sample_ID', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'total_counts_ribo', 'pct_counts_ribo', 'doublet_scores', 'predicted_doublets', 'percent_chrY', 'XIST-counts', 'XIST-percentage', 'gender_check_cov', 'S_score', 'G2M_score', 'phase'
    var: 'mt', 'ribo', 'n_cells_by_counts', 'mean_counts', 'pct_dropout_by_counts', 'total_counts'
    uns: 'Sample_ID_colors'
    layers: 'raw_counts'

## Check if data is raw or Normalized

In [4]:
def X_is_raw(adata):
    return np.array_equal(adata.X.sum(axis=0).astype(int), adata.X.sum(axis=0))

In [5]:
adata.X = adata.layers['raw_counts']

In [6]:
print(X_is_raw(adata))

True


# QC

## Highly Variable Genes removal

In [7]:
sc.pp.highly_variable_genes(adata, flavor = "seurat_v3",n_top_genes = 8000,batch_key = 'Sample_ID', layer= 'raw_counts')
adata

extracting highly variable genes
--> added
    'highly_variable', boolean vector (adata.var)
    'highly_variable_rank', float vector (adata.var)
    'means', float vector (adata.var)
    'variances', float vector (adata.var)
    'variances_norm', float vector (adata.var)


AnnData object with n_obs × n_vars = 157012 × 32285
    obs: 'sample', 'Sample_Name', 'Sex', 'Genotype', 'Treatment', 'Condition', 'Sample_ID', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'total_counts_ribo', 'pct_counts_ribo', 'doublet_scores', 'predicted_doublets', 'percent_chrY', 'XIST-counts', 'XIST-percentage', 'gender_check_cov', 'S_score', 'G2M_score', 'phase'
    var: 'mt', 'ribo', 'n_cells_by_counts', 'mean_counts', 'pct_dropout_by_counts', 'total_counts', 'highly_variable', 'highly_variable_rank', 'means', 'variances', 'variances_norm', 'highly_variable_nbatches'
    uns: 'Sample_ID_colors', 'hvg'
    layers: 'raw_counts'

## Normalization

In [8]:
sc.pp.normalize_total(adata, target_sum=1e6)
sc.pp.log1p(adata)

normalizing counts per cell


    finished (0:00:00)


In [9]:
print(X_is_raw(adata))

False


In [10]:
adata.layers['cpm_normalization'] = adata.X.copy()

# Data visualization

In [11]:
sc.tl.pca(adata,  n_comps=50, use_highly_variable=True)
sc.pp.neighbors(adata)

computing PCA


    with n_comps=50


/home/gruengroup/srivalli/miniconda3/envs/scanpy/lib/python3.13/site-packages/scanpy/preprocessing/_pca.py:374: FutureWarning: Argument `use_highly_variable` is deprecated, consider using the mask argument. Use_highly_variable=True can be called through mask_var="highly_variable". Use_highly_variable=False can be called through mask_var=None
  warn(msg, FutureWarning)


    finished (0:00:32)
computing neighbors
    using 'X_pca' with n_pcs = 50
    finished: added to `.uns['neighbors']`
    `.obsp['distances']`, distances for each pair of neighbors
    `.obsp['connectivities']`, weighted adjacency matrix (0:00:29)


In [12]:
sc.tl.umap(adata)

computing UMAP


    finished: added
    'X_umap', UMAP coordinates (adata.obsm)
    'umap', UMAP parameters (adata.uns) (0:00:57)


In [14]:
sc.pl.umap(adata, color = ['sample','Sample_ID','Sex', 'Genotype', 'Treatment', 'Condition', 'total_counts','n_genes_by_counts', 'doublet_scores', 'predicted_doublets', 'pct_counts_mt', 'pct_counts_ribo', 'percent_chrY', 'XIST-percentage', 'gender_check_cov', 'phase'], layer= 'cpm_normalization',frameon = False, cmap= 'RdYlBu_r')

In [ ]:
sc.pl.umap(adata,color= ['Ttn','Myh6','Dcn','Col1a1','Pecam1','Cdh5','Myh11','Acta2'],frameon = False , cmap= 'RdYlBu_r',layer = 'cpm_normalization')

# Batch correction - Sample level

In [ ]:
bbknn_donor = bbknn.bbknn(adata,batch_key = 'sample', neighbors_within_batch = 4, approx = True,  copy = True)
bbknn_donor

## Data visualization after batch correction

In [ ]:
sc.tl.umap(bbknn_donor)
sc.pl.umap(bbknn_donor, color = ['sample','Sample_ID','Sex', 'Genotype', 'Treatment', 'Condition', 'total_counts','n_genes_by_counts', 'doublet_scores', 'predicted_doublets', 'pct_counts_mt', 'pct_counts_ribo', 'percent_chrY', 'XIST-percentage', 'gender_check_cov', 'phase'], layer= 'cpm_normalization',frameon = False, cmap= 'RdYlBu_r')

In [ ]:
sc.pl.umap(bbknn_donor,color= ['Ttn','Myh6','Dcn','Col1a1','Pecam1','Cdh5','Myh11','Acta2'],frameon = False,cmap= 'RdYlBu_r',layer = 'cpm_normalization')

# Batch correction - Treatment level

In [ ]:
bbknn_donor2 = bbknn.bbknn(adata,batch_key = 'Treatment', neighbors_within_batch = 4, approx = True,  copy = True)
bbknn_donor2

## Data visualization after batch correction

In [ ]:
sc.tl.umap(bbknn_donor2)
sc.pl.umap(bbknn_donor2, color = ['sample','Sample_ID','Sex', 'Genotype', 'Treatment', 'Condition', 'total_counts','n_genes_by_counts', 'doublet_scores', 'predicted_doublets', 'pct_counts_mt', 'pct_counts_ribo', 'percent_chrY', 'XIST-percentage', 'gender_check_cov', 'phase'], layer= 'cpm_normalization',frameon = False, cmap= 'RdYlBu_r')

In [ ]:
sc.pl.umap(bbknn_donor2,color= ['Ttn','Myh6','Dcn','Col1a1','Pecam1','Cdh5','Myh11','Acta2'],frameon = False,cmap= 'RdYlBu_r',layer = 'cpm_normalization')